In [ ]:
import dedalus.public as de
import numpy as np
import matplotlib.pyplot as plt
import file_tools as flt
import glob
import pandas as pd
import interpolation as ip
import matplotlib as mpl
import ternary
from scipy.optimize import fsolve
mpl.rcParams['pcolor.shading'] = 'gouraud'

# Ternary plots

In [ ]:
def angle_to_surf(θs): return np.sin(θs)/np.sum(np.sin(θs))

def theta_eqs(θs,*args):
    θ1, θ2, θ3 = θs
    σ23, σ13, σ12 = args[0]
    return np.array([np.sin(θ1)/σ23 - np.sin(θ2)/σ13,
                     np.sin(θ1)/σ23 - np.sin(θ3)/σ12,
                     θ1 + θ2 + θ3 - 2*np.pi])

def theta_eqs_jac(θs,*args):
    θ1, θ2, θ3 = θs
    σ23, σ13, σ12 = args[0]
    return np.array([[np.cos(θ1)/σ23, np.cos(θ1)/σ23, 1],
                     [- np.cos(θ2)/σ13, 0, 1],
                     [0, - np.cos(θ3)/σ12, 1]])

def surf_to_theta(σs):
    closest = np.argmin(np.sum((σs[None,:] - σs_lib)**2,axis=1))
    closest_θ = θs_lib[closest]
    return fsolve(theta_eqs, closest_θ, args=(σs,), fprime=theta_eqs_jac)
delta = .01
θs_lib = np.array([(θ1, θ2, 2*np.pi-θ1-θ2) 
                   for θ1 in np.arange(delta,np.pi-delta,delta)
                   for θ2 in np.arange(max(np.pi-θ1+delta,delta),np.pi-delta,delta)])
σs_lib = np.array([angle_to_surf(θs) for θs in θs_lib])

import matplotlib
matplotlib.rcParams['figure.dpi'] = 200
# matplotlib.rcParams['figure.figsize'] = (4, 4)

from ternary.helpers import project_point, planar_to_coordinates
surf_to_cart = project_point
cart_to_surf = lambda x: planar_to_coordinates(x, 1)

def rotate_clock(v,θ):
    return np.array([[ np.cos(θ), np.sin(θ)],
                     [-np.sin(θ), np.cos(θ)]]) @ v

def plot_angle(tax, σs, length = 0.04, **kwargs):
    dic = {}
    dic['px'] = px = surf_to_cart(σs)
    dic['σs'] = σ23, σ13, σ12 = σs
    dic['θs'] = θ1, θ2, θ3 = surf_to_theta(σs)
    dic['vx0'] = vx0 = length*np.array([0,1])
    dic['vx13'] = vx13 = vx0
    dic['vx23'] = vx23 = rotate_clock(vx0,θ3)
    dic['vx12'] = vx12 = rotate_clock(vx0,-θ1)
    dic['px13'] = px13 = px + (σ13/.5)*vx13
    dic['px23'] = px23 = px + (σ23/.5)*vx23
    dic['px12'] = px12 = px + (σ12/.5)*vx12
    dic['pc13'] = pc13 = cart_to_surf(px13)
    dic['pc23'] = pc23 = cart_to_surf(px23)
    dic['pc12'] = pc12 = cart_to_surf(px12)
#     tax.line(σs, pc12, color='C2',linewidth=1)#**kwargs)
#     tax.line(σs, pc13, color='C0',linewidth=1)#**kwargs)
#     tax.line(σs, pc23, color='C3',linewidth=1)#**kwargs)
    tax.ax.add_patch(arc_patch(px,length,np.pi/2,np.pi/2+θ1,color='C4',linewidth=.2))
    tax.ax.add_patch(arc_patch(px,length,np.pi/2,np.pi/2-θ3,color='lightgray',linewidth=.2))
    tax.ax.add_patch(arc_patch(px,length,np.pi/2+θ1,np.pi/2+θ1+θ2,color='C1',linewidth=.2))
    return tax, dic

def plot_circle(tax, σs, r, color, fill=True, n=100, **kwargs):
    px = surf_to_cart(σs)
    circle = plt.Circle(px,r,color=color,fill=fill,**kwargs)
    tax.ax.add_patch(circle)
    return tax

def plot_triangle(tax, σs, color, fill=True, n=100, **kwargs):
    pxs = [surf_to_cart(σsi) for σsi in σs]
    triangle = plt.Polygon(pxs,color=color,fill=fill, **kwargs)
    tax.ax.add_patch(triangle)
    return tax

import matplotlib.patches as mpatches
def arc_patch(center, radius, theta1, theta2, resolution=100, **kwargs):
    # generate the points
    theta = np.linspace(theta1, theta2, resolution)
    arc = radius*np.hstack((np.cos(theta),np.sin(theta)))
    points = [center]+[(center[0]+radius*np.cos(θ), center[1]+radius*np.sin(θ)) for θ in np.linspace(theta1,theta2,resolution)]
    poly = mpatches.Polygon(points, closed=True, **kwargs)
    return poly

def circle_segment(α, l, dim=3):
    if α == 0:
        out = {'α':α, 'l':l}
        if dim == 2:  out['c'], out['a'], out['r'], out['h'] = l, 0, np.inf, -np.inf
        elif dim ==3: out['a'], out['v'], out['r'], out['h'] = np.pi*l**2, 0, np.inf, -np.inf
        return out
    β = np.abs(α)
    h = -l/np.tan(β)
    r =  l/np.sin(β)
    θ0 = np.pi/2 - β
    out = {'α':α, 'l':l, 'r':r, 'h':h, 'θ0':θ0}
    # segment
    if dim == 2:
        out['c'] = c = 2*β*r
        out['a'] = a = r**2 * (β - np.sin(β)*np.cos(β)) # area
    # cap
    elif dim == 3:
        out['a'] = a = 2*np.pi*r**2 * (1 - np.cos(β))
        out['v'] = v = (np.pi/3)*r**3 * (2 + np.cos(β))*(1 - np.cos(β))**2
    return out

def spherical_cap_path(α, θ1, r, h, n=101):
    ts = np.linspace(0,1,n)
    xs = r*np.cos(θ1 + (np.pi-2*θ1)*ts)
    ys = np.sign(α)*(h + r*np.sin(θ1 + (np.pi-2*θ1)*ts))
    return xs, ys

σfunc = lambda θ1, θ2 : angle_to_surf([θ1,θ2,2*np.pi-θ1-θ2])

def fix_vol_crescent(α, θ1, θ2, dim=3):
    θs = [α,α-θ1,α-θ1-θ2]
    segments = {i: circle_segment(θs[i],1,dim=dim) for i in range(3)}
    if dim == 2:
        A = sum(segments[i]['a'] for i in [0,2])
        factor = A**(-1/2)
        for i in segments:
            segments[i]['c'] *= factor
            segments[i]['a'] *= factor**2
        segments['a'] = [segments[0]['a'] - np.sign(θs[1])*segments[1]['a'],
                         segments[2]['a'] + np.sign(θs[1])*segments[1]['a']]
    elif dim == 3:
        V = sum(segments[i]['v'] for i in [0,2])
        factor = V**(-1/3)
        for i in segments:
            segments[i]['a'] *= factor**2
            segments[i]['v'] *= factor**3
        segments['v'] = [segments[0]['v'] - np.sign(θs[1])*segments[1]['v'],
                         segments[2]['v'] + np.sign(θs[1])*segments[1]['v']]
    for i in range(3):
        segments[i]['r'] *= factor
        segments[i]['h'] *= factor
        segments[i]['l'] *= factor
    return segments

def crescent_regions(α, θ1, θ2, dim=3, transform_points=None, fix_size=False):
    segments = fix_vol_crescent(α, θ1, θ2, dim=dim)
    curves = {}
    for i in range(3):
        α, θ0, r, h = [segments[i][_] for _ in ['α','θ0','r','h']]
        curves[i] = spherical_cap_path(α, θ0, r, h)
    boundaries = {1: (np.concatenate([curves[0][0],curves[1][0][::-1]]),
                      np.concatenate([curves[0][1],curves[1][1][::-1]])),
                  2: (np.concatenate([curves[1][0],curves[2][0][::-1]]),
                      np.concatenate([curves[1][1],curves[2][1][::-1]]))}
    if not (transform_points == None):
        x0, y0, r = transform_points
        ymax = boundaries[1][1].max()
        ymin = boundaries[2][1].min()
        ymid = (ymax + ymin)/2
        if fix_size: r /= (ymax-ymin)/2
        transform_points = lambda x, y: (x*r + x0, (y-ymid)*r + y0)
        for i in boundaries:
            boundaries[i] = transform_points(*boundaries[i])
    
    shapes = {}
    for i in boundaries:
        x, y = boundaries[i]
        points = np.vstack((x,y)).T
        shapes[i] = mpatches.Polygon(points, closed=True, color='C4' if i==1 else 'C1',linewidth=.2)
    return shapes, segments

from scipy.interpolate import interp1d

# connection between angle and volume!
def angle_to_volume(θ1, θ2, dim=3, n=101):
    αs = [α for α in np.linspace(0, np.pi,n) if (α-θ1-θ2 > -np.pi) and (α-θ1-θ2 < 0)]
    segments = [fix_vol_crescent(α, θ1, θ2, dim=dim) for α in αs]
    if dim == 2:
        vs = [segment['a'] for segment in segments]
    elif dim == 3:
        vs = [segment['v'] for segment in segments]
    return interp1d(np.array(vs)[:,0], αs, fill_value=(αs[0],αs[-1]), bounds_error=False)

def plot_crescent(tax, σs, cs, rad, dim=3, transform_points=None):
    θ1,θ2,θ3 = surf_to_theta(σs)
    αhalf = angle_to_volume(θ1, θ2, dim=dim)(cs[0])
    if transform_points == None:
        transform_points = tuple(surf_to_cart(σs)) + (rad,)
    shapes, segments = crescent_regions(αhalf, θ1, θ2, dim=dim, transform_points=transform_points)
    tax.ax.add_patch(shapes[1])
    tax.ax.add_patch(shapes[2])
    return tax


from matplotlib.gridspec import GridSpec

g = lambda x: x**2 * (1-x)**2
def free_energy_local(cs, χs):
    return sum(g(c)*χ for c, χ in zip(cs, χs))

def concentration_figures(σs, c1, transform_points=(0,0,1), dim=3, fix_size=True, shape=None):
    χs = M@σs
    if shape == None:
        if min(χs) > 0: shape = 'crescent'
        elif χs[0] < 0: shape = '2 in 1'
        elif χs[1] < 0: shape = '1 in 2'
        elif χs[2] < 0: shape = 'separate'
    if shape == 'crescent':
        θ1,θ2,θ3 = surf_to_theta(σs)
        α = angle_to_volume(θ1, θ2, dim=dim)(c1)
        shapes, segments = crescent_regions(α, θ1, θ2, dim=dim, transform_points=transform_points, fix_size=fix_size)
        return shapes[1], shapes[2]
    elif shape == '2 in 1':
        center = transform_points[:2]
        r1 = transform_points[2]
        r2 = r1*(1-c1)**(1/dim)
        return plt.Circle(center, r1, color='C4',linewidth=.1), plt.Circle(center, r2, color='C1',linewidth=.1)
    elif shape == '1 in 2':
        center = transform_points[:2]
        r2 = transform_points[2]
        r1 = r2*c1**(1/dim)
        return plt.Circle(center, r2, color='C1',linewidth=.1), plt.Circle(center, r1, color='C4',linewidth=.1)
    elif shape == 'separate':
        gamma = (c1/(1-c1))**(1/3)
        r2 = transform_points[2]/(1+gamma)
        r1 = gamma*r2
        center1 = np.array(transform_points[:2]) + np.array([0,transform_points[2]-r1])
        center2 = np.array(transform_points[:2]) - np.array([0,transform_points[2]-r2])
        return plt.Circle(center2, r2, color='C1',linewidth=.1), plt.Circle(center1, r1, color='C4',linewidth=.1)

def sphere_energy(σs, c1, inner=None, dim=3): # assuming total volume = 1
    σ23, σ13, σ12 = σs
    χs = M@σs
    dic = {}
    if dim == 3:
        if inner == None:
            if χs[0] < 0: inner = 2
            elif χs[1] < 0: inner = 1
            elif χs[2] < 0: inner = (1,2)
        if inner == 1:
            inner = c1
            σin, σout = σ12, σ23
            r_o = (3/(4*np.pi))**(1/3)
            r_i = r_o*inner**(1/3)
            a_o = 4*np.pi*r_o**2
            a_i = a_o*inner**(2/3)
        elif inner == 2:
            inner = 1-c1
            σin, σout = σ12, σ13
            r_o = (3/(4*np.pi))**(1/3)
            r_i = r_o*inner**(1/3)
            a_o = 4*np.pi*r_o**2
            a_i = a_o*inner**(2/3)
        elif inner == (1,2):
            inner = c1
            σin, σout = σ13, σ23
            r_o = ((3/(4*np.pi))*(1-inner))**(1/3)
            r_i = ((3/(4*np.pi))*(inner))**(1/3)
            a_o = 4*np.pi*r_o**2
            a_i = 4*np.pi*r_i**2
        return a_i*σin + a_o*σout
# this has problems if any σ = .5

def surface_energy(σs, c1, dim=3, inner=None):
    χs = M@σs
    if min(χs) >=0 and not inner: 
        σ23, σ13, σ12 = σs
        θ1,θ2,θ3 = surf_to_theta(σs)
        α = angle_to_volume(θ1, θ2, dim=dim)(c1)
        dic = fix_vol_crescent(α, θ1, θ2, dim=dim)
        return dic[0]['a']*σ13 + dic[1]['a']*σ12 + dic[2]['a']*σ23
    else: 
        return sphere_energy(σs, c1, dim=dim, inner=inner)
    
def plot_point(p,**figkw):
    ax.scatter([p[0]],[p[1]],**figkw)
    
def plot_line(start, end, ax,**figkw):
    ax.plot([start[0],end[0]],[start[1],end[1]],linewidth=1,**figkw)
    
def plot_vector(start, angle, length, ax, arrow=False,**figkw):
    unit = np.array([1,0])
    rotation = np.array([[np.cos(angle),np.sin(angle)],
                         [-np.sin(angle),np.cos(angle)]])
    new = length*rotation.T@unit
    end = np.array(start) + new
    if not arrow: ax.plot([start[0],end[0]],[start[1],end[1]],**figkw)
    else: ax.arrow(start[0],start[1],end[0]-start[0],end[1]-start[1],**figkw)
    
def plot_arc(center, radius, theta1, theta2, ax, resolution=100, **figkw):
    # generate the points
    theta = np.linspace(theta1, theta2, resolution)
    arc = radius*np.vstack((np.cos(theta),np.sin(theta)))
    ax.plot(arc[0]+center[0],arc[1]+center[1],**figkw)
#     points = [(center[0]+radius*np.cos(θ), center[1]+radius*np.sin(θ)) for θ in np.linspace(theta1,theta2,resolution)]
#     poly = mpatches.Polygon(points, closed=True, **kwargs)    
    
M = np.array([[-1,1,1],
              [1,-1,1],
              [1,1,-1]])

# Crescent diagram

In [ ]:
α = 5*np.pi/8
θ1 = .8*np.pi
θ2 = 1.2*np.pi/2
data = fix_vol_crescent(α,θ1,θ2)
patches = crescent_regions(α,θ1,θ2)
l = patches[1][0]['l']
h0 = patches[1][0]['h']
h1 = patches[1][1]['h']
h2 = patches[1][2]['h']

fig, ax = plt.subplots()
plot_line([-l,0],[l,0],ax,color='k',linestyle='--')
plot_line([0,h0],[l,0],ax,color='k',linestyle='--')
plot_line([0,-h1],[l,0],ax,color='k',linestyle='--')
plot_line([0,-h2],[l,0],ax,color='k',linestyle='--')
plot_line([0,-h2],[0,-h1],ax,color='k',linestyle='--')
plot_point([l,0],color='k',s=5,zorder=10)
plot_point([0,0],color='k',s=5,zorder=10)
plot_point([0,h0],color='k',s=5,zorder=10)
plot_point([0,-h1],color='k',s=5,zorder=10)
plot_point([0,-h2],color='k',s=5,zorder=10)
plot_arc([l,0],.15,np.pi-α,np.pi,ax,color='k',linewidth=1)
plot_arc([l,0],.05,np.pi-α,np.pi-α+θ1,ax,color='C3',linewidth=1)
plot_arc([l,0],.05,np.pi-α+θ1,np.pi-α+θ1+θ2,ax,color='C0',linewidth=1)
plot_arc([l,0],.05,np.pi-α+θ1+θ2,np.pi-α+2*np.pi,ax,color='C2',linewidth=1)
plot_vector([l,0],np.pi-α,.2,ax,color='k',linewidth=1)
plot_vector([l,0],np.pi-α+θ1,.2,ax,color='k',linewidth=1)
plot_vector([l,0],np.pi-α+θ1+θ2,.2,ax,color='k',linewidth=1)
for patch in patches[0].values():
    ax.add_patch(patch)
fs = 7
ax.text(-0.02,-h1,'$h_3$',ha='right',va='center',fontsize=fs)
ax.text(-0.02,-h2,'$h_1$',ha='right',va='center',fontsize=fs)
ax.text(-0.02,h0,'$h_2$',ha='right',va='center',fontsize=fs)
ax.text(l/2-.05,-.02,'$\ell$',ha='center',va='top',fontsize=fs)
ax.text(l/2,-h1/2+.1,'$r_3$',ha='left',va='center',fontsize=fs)
ax.text(l/2,-h2/2-.05,'$r_1$',ha='left',va='center',fontsize=fs)
ax.text(l/2,h0/2+.05,'$r_2$',ha='left',va='center',fontsize=fs)
ax.text(l,.19,'$α$',color='C1',ha='right',va='center',fontsize=fs)
ax.text(l-.01,-.06,'$θ_2$',color='C0',ha='center',va='top',fontsize=fs,fontweight='bold')
ax.text(l-.01,.06,'$θ_1$',color='C3',ha='center',va='bottom',fontsize=fs)
ax.text(l+.06,0.0,'$θ_3$',color='C2',ha='left',va='center',fontsize=fs)
ax.set(xlim=[-1,1],ylim=[-1.25,0.75],
       xticks=[],yticks=[],
       aspect=1)
for spine in ax.spines: ax.spines[spine].set_visible(False)

# σss = np.array([[.6,.2,.2],
#                [.2,.6,.2],
#                [.2,.2,.6]])

# dy = -.22
# pos = [(-4.4,dy),(-2.6,dy),(-1.2,dy)]

# points = pos[:-1] + [[-1.2,.4+dy],[-1.2,-.4+dy]]
# for point in points: plot_point(point,color='k',s=5,zorder=10)

# for i, σs in enumerate(σss):
#     for shape in concentration_figures(σs, .5, fix_size=True, 
#                                        transform_points=(pos[i][0],pos[i][1],.8)):
#         ax.add_patch(shape)

# plt.savefig('figures/crescent-diagram.pdf',bbox_inches='tight')
# ax.text(-1.8,4,'Concentration',rotation='vertical',ha='center',va='center',fontsize=20)


# Triple ternary plot

In [ ]:
σs0 = np.array([1,1,1])/3

dσs = .1*np.array([[1,0,-1],
                   [-1,1,0],
                   [0,-1,1]])

dσsoff = .1*(1/3)*np.array([[-2,1,1],
                            [1,-2,1],
                            [1,1,-2]])

diffs = [[i,j,k] for i in [-1,0,1] for j in [-1,0,1] for k in [-1,0,1]]

plot_σs1 = σs0 + diffs@dσs
plot_σs2 = np.concatenate([σs0 + dσsoff[0] + diffs@dσs,
                           [σs0 + dσsoff[2] + 2*dσs[0] + dσs[1],
                            σs0 + dσsoff[2] + dσs[0] + 3*dσs[2]]])
plot_σs1 = plot_σs1[plot_σs1.max(axis=1)<.5]
plot_σs2 = plot_σs2[plot_σs2.max(axis=1)<.5]

In [ ]:
θ1s = np.pi*np.random.random(500)
θ2s = np.pi*np.random.random(500)
θ3s = 2*np.pi - θ1s - θ2s
θs = np.array([θ1s,θ2s,θ3s])
θs = θs[:,θs[2]<np.pi].T
σs = np.array([angle_to_surf(θi) for θi in θs])
θs2 = np.array([surf_to_theta(σi) for σi in σs])

In [ ]:
# figure measurements
θ2 = 20*np.pi/180

θ1 = np.linspace(np.pi-θ2, np.pi, 7+2)[1:-1]

θ3 = 2*np.pi - θ1 - θ2
θs = np.array([θ1,θ2+0*θ1,θ3]).T
σss = np.array([angle_to_surf([θ1[i],θ2,θ3[i]]) for i in range(len(θ1))])

M = np.array([[-1,1,1],
              [1,-1,1],
              [1,1,-1]])

import pandas as pd

df = pd.DataFrame(σss[:,::-1])

df.columns = ['σ12','σ13','σ23']

# df.to_csv('hpc/surface-tensions-ninth.csv', index=None)

from matplotlib.cm import RdYlGn

surfcolours = RdYlGn(np.linspace(.1,.9,len(σss)))

In [ ]:
# three plots
fig, axes = plt.subplots(2,3,gridspec_kw={'hspace':0.2,'wspace':.05,'height_ratios':[1,.05]},
                       figsize=(11,4))
ax = {(0,0):axes[0,0],(1,0):axes[0, 1],(1,1):axes[0, 2]}
# plot angles
fig, tax = ternary.figure(ax=ax[0,0])
fig.set_facecolor('w')
tax.clear_matplotlib_ticks()
tax.get_axes().axis('off')
tax.boundary(linewidth=1.5)
tax.gridlines(color="gray", multiple=.1, zorder=0)

fontsize = 14
# tax.set_title("Surface tension vs Contact angles\n", fontsize=fontsize)
tax.bottom_axis_label("$\sigma_{2,3}$", fontsize=fontsize, offset=.12, color='C3')
tax.right_axis_label("$\sigma_{1,3}$      ", fontsize=fontsize, offset=.2, color='C0')
tax.left_axis_label("      $\sigma_{1,2}$", fontsize=fontsize, offset=.2, color='C2')
locs = list(np.arange(.1,.91,.2))

tax.ticks(axis='l', multiple=.2, linewidth=1, offset=.04, tick_formats="%.1f")
tax.ticks(axis='b', multiple=.2, linewidth=1, offset=.03, tick_formats="%.1f")
tax.ticks(axis='r', multiple=.2, linewidth=1, offset=.03, tick_formats="%.1f")

tax.set_title('Surface tension',fontsize=14,pad=5)
tax.set_background_color(color="lightgray") # the detault, essentially
tax.horizontal_line(.5,color='C0',linewidth=1.5, zorder=-1)
tax.left_parallel_line(.5,color='C3',linewidth=1.5, zorder=-1)
tax.right_parallel_line(.5,color='C2',linewidth=1.5, zorder=-1)

for σi in np.concatenate([plot_σs1, plot_σs2]):
    plot_angle(tax, σi, length=.02)

circles = [((.1,.25,.65), .05, 'C4'),
           ((.25,.1,.65), .05, 'C1'),
           ((1/6,2/3,1/6), .05*2**(1/2), 'C1'),
           ((1/6,2/3,1/6), .05, 'C4'),
           ((2/3,1/6,1/6), .05*2**(1/2), 'C4'),
           ((2/3,1/6,1/6), .05, 'C1')]

for circle in circles: plot_circle(tax, *circle)

tax.scatter(σss,color=surfcolours,cmap='RdYlGn',marker='*',zorder=10)

tax._redraw_labels()
ax[0,0].set(aspect=1,ylim=[-.1,1])

# plot 3D crescents
ii = list(range(len(σss)))
cs = np.arange(.1,.91,.1)
for j, i in enumerate(ii[::-1]):
    for k, c in enumerate(cs):
        dim = 3
        for shape in concentration_figures(σss[i], c, transform_points=(j,-k,.4), fix_size=True, dim=dim):
            ax[1,0].add_patch(shape)
for spine in ax[1,0].spines: ax[1,0].spines[spine].set_visible(False)

ax[1,0].scatter(np.arange(len(σss)),.7*np.ones(len(σss)),color=surfcolours[::-1],marker='*')

yticklabels = [(f'{int(round(10*(1-c1))):d}',f'{int(round(10*c1)):d}') for c1 in cs]
xticklabels = [(f'{σs[2]:.3f}'[1:],f'{σs[0]:.3f}'[1:]) for σs in σss[::-1]]
yticklabels = [((-.8,i-8+.03,ytick[0],{'color':'C4','ha':'center','va':'bottom', 'fontsize':8}),
                (-.8,i-8,'_',{'ha':'center','va':'bottom', 'fontsize':8}),
                (-.8,i-8-.05,ytick[1],{'color':'C1','ha':'center','va':'top', 'fontsize':8})) for i, ytick in enumerate(yticklabels)]
xticklabels = [((i, -8-.8, xtick[0], {'color':'C2', 'ha':'center', 'va':'center', 'fontsize':8}),
                (i, -8-1.3, xtick[1], {'color':'C3', 'ha':'center', 'va':'center', 'fontsize':8})) for i, xtick in enumerate(xticklabels)]

# ax.set(xlim=[-.5,12.5], ylim=[-.5,8.5],aspect=1)
ax[1,0].text(-1.3,-8+4.1,'$c_1$',color='C4',ha='center',va='bottom',fontsize=8,weight='bold')
ax[1,0].text(-1.3,-8+4,'_',ha='center',va='bottom',fontsize=8)
ax[1,0].text(-1.3,-8+3.95,'$c_2$',color='C1',ha='center',va='top',fontsize=8)
ax[1,0].text(-1.4,-8-.85, '$\sigma_{1,2}$',color='C2',ha='left',fontsize=8)
ax[1,0].text(-1.4,-8-1.35, '$\sigma_{2,3}$',color='C3',ha='left',fontsize=8,clip_on=False,zorder=10)
for text in yticklabels+xticklabels:
    for s in text:
        ax[1,0].text(s[0], s[1], s[2], **s[3])
# ax.set_title('Surface tensions and Concentrations vs Morphology',fontsize=20)

ax[1,0].text(-2,-4,'Concentration',rotation='vertical',ha='center',va='center',fontsize=14)
ax[1,0].text(3,-10,'Surface Tensions',ha='center',va='center',fontsize=10)
ax[1,0].set_title('  Particle shape',fontsize=14,pad=5)

ax[1,0].set(xlim=[-1-.5,len(ii)-.5],
            ylim=[-2-len(cs)+2.5,1],
            xticks=[],yticks=[],aspect=1)


σs = σs0
M = np.array([[-1,1,1],
              [1,-1,1],
              [1,1,-1]])
χs = M@σs

f = lambda cs: free_energy_local(cs, χs)

# plot experimental free energy
ffig, fax = ternary.figure(ax=ax[1,1])
ffig.set_facecolor('w')
fax.clear_matplotlib_ticks()
fax.get_axes().axis('off')
fax.boundary(linewidth=1.5)
fax.set_background_color('k', alpha=1)
# stax.gridlines(color="gray", multiple=.1, zorder=0)

fax.set_title(f"Free energy surface", fontsize=14,pad=5)
fax.bottom_axis_label("$c_1$", fontsize=fontsize, offset=.1, color='C4')
fax.right_axis_label("$c_2$      ", fontsize=fontsize, offset=.2, color='C1')
fax.left_axis_label("      $c_3$", fontsize=fontsize, offset=.2, color='darkgray')
fax._redraw_labels()
fax.ticks(axis='l', multiple=.2, linewidth=1, offset=.04, tick_formats="%.1f")
fax.ticks(axis='b', multiple=.2, linewidth=1, offset=.03, tick_formats="%.1f")
fax.ticks(axis='r', multiple=.2, linewidth=1, offset=.03, tick_formats="%.1f")

σexp = σss[0]
χexp = M@σexp
# poss = [(-.05,-.12),(.5,-.12),(1.05,-.12)]
poss = [(-.05,.9),(-.05,.8),(-.05,.7)]
texts = [(*poss[0],f'$\sigma_{{1,2}} = {σexp[2]:.3f}$',{'color':'C2','ha':'left'}),
         (*poss[1],f'$\sigma_{{1,3}} = {σexp[1]:.3f}$',{'color':'C0','ha':'left'}),
         (*poss[2],f'$\sigma_{{2,3}} = {σexp[0]:.3f}$',{'color':'C3','ha':'left'})]
for text in texts: fax.ax.annotate(text[2],(text[0],text[1]),xycoords='axes fraction',**text[3])
pcs = np.array([np.array(tri)/50 for tri in ternary.helpers.simplex_iterator(50)])
pcxs = np.array([surf_to_cart(cs) for cs in pcs])

fs = free_energy_local(pcs.T, χexp)
maxloc = pcxs[np.argmax(fs)]
free_energy_plot = fax.ax.tripcolor(pcxs[:,0], pcxs[:,1], fs, shading='gouraud', cmap='inferno',zorder=-2)
fax.ax.tricontour(pcxs[:,0], pcxs[:,1], fs, colors='k', linewidths=.1,zorder=-2)
fax.ax.scatter(*maxloc, c='k', marker='o')
fax.ax.scatter([.28],[.96],c='C3',marker='*')
fax.ax.set(aspect=1,ylim=[-.1,1])
plt.colorbar(free_energy_plot, cax=axes[1,2], orientation='horizontal')
axes[1,2].xaxis.set_ticks_position('top')
# axes[1,2].xaxis.set_tick_params(pad=-2)

ax[0,0].annotate('$(a)$',(-0,1.03),xycoords='axes fraction',ha='left',fontsize=fontsize)
ax[1,0].annotate('$(b)$',(-0,1.025),xycoords='axes fraction',ha='left',fontsize=fontsize)
# ax[0,1].annotate('$(c)$',(-0,1),xycoords='axes fraction',ha='left',fontsize=fontsize)
ax[1,1].annotate('$(c)$',(-0.05,1.03),xycoords='axes fraction',ha='left',fontsize=fontsize)
ax[1,1].annotate('Dimensionless Free Energy',(.5,-.22),xycoords='axes fraction',ha='center',fontsize=10)

fig.delaxes(axes[1,0])
fig.delaxes(axes[1,1])
# for spine in ax[2,0].spines:
#     ax[2,0].spines[spine].set_visible(False)
# ax[2,0].set(xticks=[],yticks=[])
# fig.savefig(f'figures/ternary-plots/experimental-crescents-vertical.pdf',bbox_inches='tight')

fig.savefig('figures/surface-tension-shape-free-energy-diagram.png',bbox_inches='tight',dpi=400)

# Crescent schematic

In [ ]:
θ1, θ2, θ3 = θs[0]
α = .15*np.pi
data = fix_vol_crescent(α,θ1,θ2)
patches = crescent_regions(α,θ1,θ2)
l = patches[1][0]['l']
h0 = patches[1][0]['h']
h1 = patches[1][1]['h']
h2 = patches[1][2]['h']

fig, ax = plt.subplots()
plot_point([l,0],color='k',s=5,zorder=12)
plot_arc([l,0],.04,np.pi-α,np.pi-α+θ1,ax,color='purple',linewidth=1)
plot_arc([l,0],.04,np.pi-α+θ1,np.pi-α+θ1+θ2,ax,color='brown',linewidth=1)
plot_arc([l,0],.04,np.pi-α+θ1+θ2,np.pi-α+2*np.pi,ax,color='gray',linewidth=1)
plot_vector([l,0],np.pi-α,.6*σss[0,1],ax,color='C0',linewidth=1,head_width=.02,width=.002,arrow=True,zorder=10)
plot_vector([l,0],np.pi-α+θ1,.6*σss[0,2],ax,color='C2',linewidth=1,head_width=.02,width=.002,arrow=True,zorder=10)
plot_vector([l,0],np.pi-α+θ1+θ2,.6*σss[0,0],ax,color='C3',linewidth=1,head_width=.02,width=.002,arrow=True,zorder=10)
for patch in patches[0].values():
    ax.add_patch(patch)
fs = 12
ax.text(l-.05,-.1,'$θ_1$',color='purple',ha='center',va='center',fontsize=fs)
ax.text(l+.155,-.15,'$θ_2$',color='brown',ha='center',va='center',fontsize=fs,fontweight='bold')
ax.text(l+.06,0.1,'$θ_3$',color='gray',ha='center',va='center',fontsize=fs)
ax.text(0,-.35,'Gelatin rich',color='purple',ha='center',fontsize=fs)
ax.text(0,-.45,'$c_1$',color='purple',ha='center',fontsize=fs)
ax.text(0,-.9,'PEG rich',color='brown',ha='center',fontsize=fs)
ax.text(0,-1,'$c_2$',color='brown',ha='center',fontsize=fs)
# ax.text(0,.9,'$(b)$',fontsize=15,transform=ax.transAxes)
for spine in ax.spines: ax.spines[spine].set_visible(False)
texts = [(-.55,-1.5,f'$\sigma_{{1,2}} = {σss[0,2]:.3f}$',{'color':'C2','ha':'left'}),
         (-.55,-1.3,f'$\sigma_{{1,3}} = {σss[0,1]:.3f}$',{'color':'C0','ha':'left'}),
         (-.55,-1.4,f'$\sigma_{{2,3}} = {σss[0,0]:.3f}$',{'color':'C3','ha':'left'}),
         (0.1,-1.3,f'$\\theta_1 = {θ1*180/np.pi:.1f}^\circ$',{'color':'C4','ha':'left'}),
         (0.1,-1.4,f'$\\theta_2 = {θ2*180/np.pi:.1f}^\circ$',{'color':'C1','ha':'left'}),
         (0.1,-1.5,f'$\\theta_3 = {(2*np.pi-θ1-θ2)*180/np.pi:.1f}^\circ$',{'color':'gray','ha':'left'})]
for text in texts: ax.annotate(text[2],(text[0],text[1]),fontsize=10,**text[3])

ax.set(xlim=[-.7,.7],ylim=[-1.6,0.25],
       xticks=[],yticks=[],
       aspect=1)
plt.savefig('figures/crescent-schematic.png',dpi=400,bbox_inches='tight')

# Time series plot comparison

In [ ]:
# import file_tools as flt
# def mag(x): return np.log10(np.abs(x)+1e-16)
# import glob
# from mpi4py import MPI
# import time
# comm = MPI.COMM_WORLD
# rank, size = comm.rank, comm.size
# import matplotlib.pyplot as plt
from skfmm import distance

from math import floor, log10, ceil
# multiples = {2:log10(2), 5:log10(5)}

colors = {1: 'C4', 2: 'C1', 3: 'lightgray'}
from matplotlib.colors import to_rgb
colors = {i: np.array(to_rgb(colors[i])) for i in colors}

def format_exp(x,sig=1):
    if x == 0: return '0'
    l = np.log10(x)
    p = floor(l)
    n = str(round(10**(l - p),sig))
    if sig == 0: n = n[:-2]
    return f'{n} \\times 10^{{ {p:d} }}'

def clip(c): 
    copy = c.copy()
    copy[c<0] = 0
    copy[c>1] = 1
    return copy

def color_func(c1,c2):
    sel1 = np.s_[...,None]
    sel2 = np.s_[None,None,...]
    return clip(colors[1][sel2]*c1[sel1] + colors[2][sel2]*c2[sel1] + colors[3][sel2]*(1-c1-c2)[sel1])

def catch_int_reshape(arr, shape):
    if isinstance(arr, int): return arr + np.zeros(shape)
    else: return arr.reshape(shape)

def concat_lists(lists):
    out = []
    for l in lists:
        out.extend(l)
    return out

from scipy.interpolate import RectBivariateSpline
def uniform_interpolate_chebyshev(arr,grid_old,grid_new):
    return RectBivariateSpline(*grid_old, arr)(*grid_new,grid=True)

def list_saves(files):
    if isinstance(files[0],str): files = [files]
    sorted_files = [f for series in files for f in sorted(series, key=lambda s: int(s.split('_s')[1].split('.')[0]))]
    saves = []
    for i, f in enumerate(sorted_files):
        t, it = flt.load_data(f, 'sim_time','iteration',group='scales',asscalar=False)
        for j, (tj, itj) in enumerate(zip(t,it)):
            if not (i != 0 and itj == 0): # don't double count start of restarts
                saves.append({
                    'file_index':i, 
                    'file':f, 
                    'save_index':j, 
                    'sim_time':tj, 
                    'sim_it':itj})
    for i, save in enumerate(saves):
        save['index'] = i
    return saves

def correct_times(saves,save_series):
    for i in range(1,len(save_series)):
        pre_series, series = save_series[i-1:i+1]
        start_time = max([save['sim_time'] for save in saves if pre_series in save['file']])
        for save in saves:
            if series in save['file']:
                save['sim_time'] += start_time
    return saves

def recentre_drop(grids_old,c1,c2,extra=None):
    grids_new = [np.linspace(-1,1,len(grid)) for grid in grids_old]
    c3 = 1-c1-c2
    c1,c2,c3 = [uniform_interpolate_chebyshev(arr,grids_old,grids_new) for arr in [c1,c2,c3]]
    dist = distance(c3-.5,dx=float(grids_new[0][1]-grids_new[0][0]),periodic=True)
    minimum_ind_x = np.unravel_index(np.argmin(dist,axis=None),dist.shape)[0]
    centre = len(grids_old[0])//2
    shift = centre - minimum_ind_x
    c1, c2 = np.roll(c1,shift,axis=0), np.roll(c2,shift,axis=0)
    if not extra == None:
        extra = [uniform_interpolate_chebyshev(arr,grids_old,grids_new) for arr in extra]
        extra = [np.roll(arr,shift,axis=0) for arr in extra]
        return c1, c2, extra
    return c1, c2

def plot_concentrations(c1,c2,ax):
    color_arr = color_func(c1,c2)
    ax.imshow(color_arr.transpose(1,0,2),origin='lower',extent=[-1,1,-1,1])
    ax.set(aspect=1,xticks=[],yticks=[])
    for spine in ax.spines: ax.spines[spine].set_visible(False)
    return fig, ax 

## Time series plot

In [ ]:
# make plot dir
plot_dir = 'figures'
flt.makedir(plot_dir)
dims = 'xz'
sims = range(28)
ordering = (np.array([22,15,8,1])[None,:] + np.arange(6)[:,None]).flatten()
ref_ts = [1e-2, 1e-1, 5e-1, 10, 100, 280]
N_times = len(ref_ts)
N_sims = len(ordering)
N_series = 4
sim_dirs = {sim:{} for sim in sims}
plot_name = 'chs-2D-comparison-T-1e-2'

for sim in sims:
    d = sim_dirs[sim]
    d['series_name'] = 'chs-2D-comparison-T-1e-2' if sim < 21 else 'ch-2D-comparison-T-1e-2'
    d['save_series'] = [f'{d["series_name"]}-{i}' for i in range(10)]
    d['index'] = sim if sim < 21 else sim - 21
    d['files'] = [glob.glob(f'hpc/conservation/data/{series}/analysis-{series}-{d["index"]:0>3d}/*.h5') for series in d['save_series']]
    d['files'] = [f for f in d['files'] if f]
    d['save_series'] = [s for i, s in enumerate(d['save_series']) if i < len(d['files'])]
    d['saves'] = saves = list_saves(d['files'])
    if sim < 21: d['saves'] = correct_times(list_saves(d['files']),d['save_series'])
    d['sim_times'] = np.array([save['sim_time'] for save in d['saves']])

In [ ]:
x_old, z_old = flt.load_data(sim_dirs[0]['saves'][0]['file'],'x/1.0','z/1.0',group='scales')

fig, ax = plt.subplots(N_sims//2,2*N_times,
                       figsize=(2*N_times,N_sims//2),
                       gridspec_kw={'hspace':-0.03,'wspace':-0.03},)
for i, sim in enumerate(ordering):
    d = sim_dirs[sim]
    row = i%(N_sims//2)
    for j, t in enumerate(ref_ts):
        if t >= 0: it = np.argmin(np.abs(d['sim_times'] - t))
        else: it = -1
        col = j + N_times*(i//(N_sims//2))
        save = d['saves'][it]
        c1, c2 = flt.load_data(save['file'],'c1','c2',group='tasks',sel=np.s_[save['save_index'],...])
        c1, c2 = recentre_drop((x_old,z_old),c1,c2)
        plot_concentrations(c1,c2,ax[row,col])

# dividing black lines between each concentration
for i in range(N_series-1,N_sims//2-1,N_series):        
    for j in range(2*N_times):
        ax[i,j].annotate('', xy=(-0.03, 0.03), xycoords='axes fraction', 
                         xytext=(1.03, .03), arrowprops=dict(arrowstyle="-", color='k'),
                         zorder=10,annotation_clip=False)
# dividing black lines between left and right
for i in range(N_sims//2):
    ax[i,N_times].annotate('', xy=(0.03, -0.03), xycoords='axes fraction', 
                     xytext=(.03, 1.03), arrowprops=dict(arrowstyle="-", color='k'),
                     zorder=10,annotation_clip=False)
# all the text to annotate
tdicts = {0:{'ha':'center','va':'bottom', 'fontsize':20, 'xycoords':'axes fraction'},
          1:{'color':'C4','ha':'center','va':'bottom', 'fontsize':20, 'xycoords':'axes fraction'},
          2:{'color':'C1', 'ha':'center', 'va':'center', 'fontsize':20, 'xycoords':'axes fraction'},
          3:{'ha':'center','va':'baseline','fontsize':12, 'xycoords':'axes fraction'},
          4:{'ha':'left','va':'center','fontsize':16, 'xycoords':'axes fraction'}}

annotations = []
for k, i in enumerate([2,6,10]):
    annotations.append(((i,0),f'{3+k}',(-1.8,.6+.5),tdicts[1]))
    annotations.append(((i,0),'$-$',     (-1.8,.4+.5),tdicts[0]))
    annotations.append(((i,0),f'{7-k}',(-1.8,.25+.5),tdicts[2]))
    annotations.append(((i,2*N_times-1),f'{6+k}',(2.6,.6+.5),tdicts[1]))
    annotations.append(((i,2*N_times-1),'$-$',     (2.6,.4+.5),tdicts[0]))
    annotations.append(((i,2*N_times-1),f'{4-k}',(2.6,.25+.5),tdicts[2]))
for j, t in enumerate(ref_ts[:-1]):
    annotations.append(((0,j),f'${format_exp(t,0)}$',(0.5,1.1),tdicts[3]))
    annotations.append(((0,j+N_times),f'${format_exp(t,0)}$',(0.5,1.1),tdicts[3]))
annotations.append(((0,N_times-1),f'End',(0.5,1.1),tdicts[3]))
annotations.append(((0,2*N_times-1),f'End',(0.5,1.1),tdicts[3]))
for i in range(0,N_sims//2,N_series):
    annotations.append(((i+1-1,0),'CH',(-1.3,.5),tdicts[4]))
    annotations.append(((i+2-1,0),'CHS',(-1.3,.5),tdicts[4]))
    annotations.append(((i+3-1,0),'CHSB z',(-1.3,.5),tdicts[4]))
    annotations.append(((i+4-1,0),'CHSB x',(-1.3,.5),tdicts[4]))
    annotations.append(((i+1-1,2*N_times-1),'CH',(1.2,.5),tdicts[4]))
    annotations.append(((i+2-1,2*N_times-1),'CHS',(1.2,.5),tdicts[4]))
    annotations.append(((i+3-1,2*N_times-1),'CHSB z',(1.2,.5),tdicts[4]))
    annotations.append(((i+4-1,2*N_times-1),'CHSB x',(1.2,.5),tdicts[4]))
annotations.append(((0,0),'Time $t$',(-1.3,1.1),{'ha':'left','va':'baseline','fontsize':16, 'xycoords':'axes fraction'}))
annotations.append(((0,N_times),'Mixed droplet evoution for CH, CHS, CHSB z, and CHSB x models at $ε = 10^{-2},T=10^{-2}$',
                    (0,1.5),{'ha':'center','fontsize':20, 'xycoords':'axes fraction'}))
for a in annotations:
    ax[a[0]].annotate(a[1],xy=a[2],annotation_clip=False,**a[3])
fig.set_facecolor('w')
plt.savefig(f'{plot_dir}/{plot_name}.pdf',bbox_inches='tight')        

## Movie frames

In [ ]:
# make plot dir
plot_dir = 'figures'
flt.makedir(plot_dir)
dims = 'xz'
sims = range(42+7)
len_series = 7
start_sims = [42,14,7,0]
N_series = len(start_sims)
series_indices = np.array([range(start,start+len_series) for start in start_sims])
N_sims = N_series*len_series
# plot_name = 'chs-2D-comparison-T-1e-2'
# save_series = [f'chs-2D-comparison-T-1e-2-{i}' for i in range(7)]

sim_dirs = {sim:{} for sim in sims}

for sim in sims:
    d = sim_dirs[sim]
    d['series_name'] = 'chs-2D-comparison-T-1e-2' if sim < 42 else 'ch-2D-comparison-T-1e-2'
    d['save_series'] = [f'{d["series_name"]}-{i}' for i in range(10)]
    d['index'] = sim if sim < 42 else sim - 42
    d['files'] = [glob.glob(f'hpc/conservation/data/{series}/analysis-{series}-{d["index"]:0>3d}/*.h5') for series in d['save_series']]
    d['files'] = [f for f in d['files'] if f]
    d['save_series'] = [s for i, s in enumerate(d['save_series']) if i < len(d['files'])]
    d['saves'] = saves = list_saves(d['files'])
    if sim < 42: d['saves'] = correct_times(list_saves(d['files']),d['save_series'])
    d['sim_times'] = np.array([save['sim_time'] for save in d['saves']])
    d['axis_indices'] = np.unravel_index(np.argmin(-1.0*np.abs(series_indices==sim)),series_indices.shape)

In [ ]:
x_old, z_old = flt.load_data(sim_dirs[0]['saves'][0]['file'],'x/1.0','z/1.0',group='scales')

longest, nt = max(((sim,len(sim_dirs[sim]['saves'])) for sim in sim_dirs),key=lambda x: x[1])

for save_index in range(nt):
    fig, ax = plt.subplots(N_series,len_series,
                           figsize=(len_series,N_series),
                           gridspec_kw={'hspace':-0.05,'wspace':-0.05},)
    for sim, sim_dir in sim_dirs.items():
        saves = sim_dir['saves']
        if save_index >= len(saves): save = saves[-1]
        else: save = saves[save_index]
        c1, c2 = flt.load_data(save['file'],'c1','c2',group='tasks',sel=np.s_[save['save_index'],...])
        c1, c2 = recentre_drop((x_old,z_old),c1,c2)
        plot_concentrations(c1,c2,ax[sim_dir['axis_indices']])
        
    ax[0,0].set(ylabel='CH')
    ax[1,0].set(ylabel='CHS')
#     ax[2,0].set(ylabel='CHNS')
    ax[2,0].set(ylabel='CHSB z')
#     ax[4,0].set(ylabel='CHNSB z')
    ax[3,0].set(ylabel='CHSB x')
#     ax[6,0].set(ylabel='CHNSB x')
    ax[0,0].set_title('$c_1:c_2 = 2:8$',fontsize=8)
    ax[0,1].set_title('$c_1:c_2 = 3:7$',fontsize=8)
    ax[0,2].set_title('$c_1:c_2 = 4:6$',fontsize=8)
    ax[0,3].set_title('$c_1:c_2 = 5:5$',fontsize=8)
    ax[0,4].set_title('$c_1:c_2 = 6:4$',fontsize=8)
    ax[0,5].set_title('$c_1:c_2 = 7:3$',fontsize=8)
    ax[0,6].set_title('$c_1:c_2 = 8:2$',fontsize=8)
    fig.suptitle(f'$t = {format_exp(sim_dirs[longest]["saves"][save_index]["sim_time"])}$',y=.99)
    fig.set_facecolor('w')
    plt.savefig(f'{plot_dir}/{plot_name}-Re-0-{save_index:0>3d}.png',dpi=200,bbox_inches='tight')
    plt.close()
    print(save_index)

## Velocity plots

In [ ]:
dx = sim_dirs[3]
dz = sim_dirs[10]

In [ ]:
its = {'x':42,'z':60}
saves = {'x':dx['saves'][its['x']], 'z':dz['saves'][its['z']]}
x, z = flt.load_data(saves['x']['file'],'x/1.0','z/1.0',group='scales')
xx, zz = np.meshgrid(x,z,indexing='ij')
arrs_old = {'x':{},'z':{}}
arrs = {'x':{},'z':{}}
for direction in 'xz':
    for name in ['c1','c2','ux','uz']:
        s = saves[direction]
        arrs_old[direction][name], = flt.load_data(s['file'],name,group='tasks',sel=s['save_index'])
    c1_old, c2_old, ux_old, uz_old = [arrs_old[direction][name] for name in ['c1','c2','ux','uz']]
    a = arrs[direction]
    a['c1'], a['c2'], (a['ux'], a['uz']) = recentre_drop((x,z),c1_old,c2_old,(ux_old,uz_old)) 

xn, zn = np.linspace(-1,1,192), np.linspace(-1,1,384)
xxn, zzn = np.meshgrid(xn,zn,indexing='ij')
speeds = {'x': ((arrs['x']['ux']-arrs['x']['ux'].mean(axis=0)[None,:])**2 + arrs['x']['uz']**2)**(1/2),
          'z': (arrs['z']['ux']**2 + arrs['z']['uz']**2)**(1/2)}

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(5.5,2),gridspec_kw={'wspace':.01})
ax = ax[None,:]
plot_concentrations(arrs['z']['c1'],arrs['z']['c2'],ax[0,0])
plot_concentrations(arrs['x']['c1'],arrs['x']['c2'],ax[0,1])
sub = np.s_[::10,::20]

ax[0,0].quiver(xxn[sub].T, zzn[sub].T, arrs['z']['ux'][sub].T, arrs['z']['uz'][sub].T,
               speeds['z'][sub].T,clim=[0,.1],cmap='Blues_r',
               angles='xy', scale_units='xy', scale=.06, width=.005)
p0 = ax[0,1].quiver(xxn[sub].T, zzn[sub].T, (arrs['x']['ux'] - 1*np.mean(arrs['x']['ux'],axis=0)[None,:])[sub].T, arrs['x']['uz'][sub].T,
                    speeds['x'][sub].T,clim=[0,.1],cmap='Blues_r',
                    angles='xy', scale_units='xy',scale=1,width=.005,)
for i in range(2):
    for spine in ax[0,i].spines:
        ax[0,i].spines[spine].set_visible(True)
    d = 'zx'[i]
    ax[0,i].set(ylabel=(f'$t = {saves[d]["sim_time"]:.0f}$'),
                title=f'CHSB ${d}$ velocity',xlabel=f'$ |u|_{{\max}} = {format_exp(np.abs(speeds[d]).max())}$')
# fig.suptitle(f'Velocity comparison CHSB $z$ vs CHSB $x$            ',y=1.03)

plt.colorbar(p0, ax=ax, orientation='vertical', pad=0.02, aspect=20,ticks=np.linspace(0,.1,5))
fig.text(.92,0.52,'Speed $|u|$',ha='center',va='center',rotation=90)
plt.savefig('figures/velocity.pdf',bbox_inches='tight')

# Cahn Hilliard ic comparison and energy comparison plot

In [ ]:
def plot_concentrations_one_ax(c1,c2,ax,loc):
    color_arr = color_func(c1,c2)
    ax.imshow(color_arr.transpose(1,0,2),origin='lower',extent=loc)

In [ ]:
def closest(arr, val):
    return np.argmin(np.abs(np.array(arr) - val))

# make plot dir                                                                                                                                            
plot_dir = f'figures'
flt.makedir(plot_dir)
sims = range(2)
sim_dirs = {sim:{} for sim in sims}
plot_name = 'ch-ic-comparison'
save_series = ['ch-ic-comparison']+[f'ch-ic-comparison-{i}' for i in range(1,5)]

for sim in sims:
    d = sim_dirs[sim]
    d['files'] = file_names = [glob.glob(f'hpc/conservation/data/{series}/analysis-{series}-{sim:0>3d}/*.h5') for series in save_series]
    d['saves'] = saves = list_saves(file_names)

x_old, z_old = flt.load_data(sim_dirs[0]['saves'][0]['file'],'x/1.0','z/1.0',group='scales')

nt = min(len(sim_dirs[sim]['saves']) for sim in sim_dirs)
# 2e-2, 5e-2,
ref_ts = [0, 1e-1, 5e-1, 2, 5, 20, 100]
N_times = len(ref_ts)
ts = [s['sim_time'] for s in sim_dirs[0]['saves']]
ref_its = [closest(ts, ti) for ti in ref_ts]

In [ ]:
θ2 = 20*np.pi/180
θ1 = np.linspace(np.pi-θ2, np.pi, 7+2)[1:-1]
θ3 = 2*np.pi - θ1 - θ2
σss = np.array([angle_to_surf([θ1[i],θ2,θ3[i]]) for i in range(len(θ1))])
M = np.array([[-1,1,1],
              [1,-1,1],
              [1,1,-1]])
σs = σss[0]
χs = M@σs
vs = np.array([.001]+list(np.linspace(0,1,101)[1:-1]) + [1-1e-3])
e1s = np.array([surface_energy(σs, v, inner=1) for v in vs])
e2s = np.array([surface_energy(σs, v, inner=2) for v in vs])
e12s = np.array([surface_energy(σs, v, inner=(1,2)) for v in vs])
ecs = np.array([surface_energy(σs, v) for v in vs])


In [ ]:
# cahn-hilliard time series figure
fig, axes = plt.subplots(2,1,figsize=(6,5),gridspec_kw={'height_ratios':[2,3],
                                                       'hspace':.3})
for j, it in enumerate(ref_its):
    for sim in sims:
        save = sim_dirs[sim]['saves'][it]
        c1, c2 = flt.load_data(save['file'],'c1','c2',group='tasks',sel=np.s_[save['save_index'],...])
        c1, c2 = recentre_drop((x_old,z_old),c1,c2)
        plot_concentrations_one_ax(c1,c2,axes[0],loc=[j,j+1,-sim-1,-sim],)
        t = save['sim_time']
axes[0].set(xlim=[0,N_times],ylim=[-2,0],aspect='auto')

annotations = []
annotate_dic = {'ha':'center','va':'baseline','fontsize':7, 'xycoords':'data'}
title_dic = annotate_dic.copy()
title_dic['fontsize'] = 12
for j, t in enumerate(ref_ts):
    if j > 0: annotations.append(((j+0.5,0.1),f'${format_exp(t,0)}$',annotate_dic))
annotations.append(((.5,.1),'Time $t = 0$',annotate_dic))
annotations.append(((N_times//2+.5,.4),'Cahn-Hilliard dependence on initial conditions',title_dic))
for a in annotations:
    axes[0].annotate(a[1],xy=a[0],annotation_clip=False,**a[2])
axes[0].set(xticks=[],yticks=[])
axes[0].annotate('$(a)$',xy=(-.1,1.1),xycoords='axes fraction',fontsize=12)
axes[0].annotate('Initial state',xy=(-.1,.5),xycoords='axes fraction',va='center',rotation='vertical',fontsize=10)
axes[0].annotate('Mixed',xy=(-.04,.75),xycoords='axes fraction',va='center',rotation='vertical',fontsize=8)
axes[0].annotate('Separate',xy=(-.04,.25),xycoords='axes fraction',va='center',rotation='vertical',fontsize=8)
axes[0].annotate('Final state',xy=(1.05,.5),xycoords='axes fraction',va='center',rotation=270,fontsize=10)
axes[0].annotate('Shell',xy=(1.01,.75),xycoords='axes fraction',va='center',rotation=270,fontsize=8)
axes[0].annotate('Crescent',xy=(1.01,.25),xycoords='axes fraction',va='center',rotation=270,fontsize=8)

# energy-volume figure
ax = axes[1]
shape_ax = ax.twinx()
shape_ax.set(xlim=[0,1],ylim=[0,1])
for c1i in np.arange(.1,.91,.2):
    for yj, shape in zip([.38,.62],['crescent', '1 in 2']):
        for patch in concentration_figures(σs,c1i,
                                           transform_points=(c1i,yj,.06),
                                           shape=shape,):
            shape_ax.add_patch(patch)
shape_ax.set(aspect=1,yticks=[])

# ax.plot(vs, e2s, label='2-in-1 shell')
# ax.plot(vs, e12s, label='1, 2 separate')
ax.plot(vs, e1s, label='1-in-2 shell', color='C3')
ax.plot(vs, ecs, label='crescent',color='C2')
ax.legend(frameon=False,loc=(.72,.37))
ax.set(xlabel='Concentration of phase one $c_{1,0}$',
       ylabel='Surface Energy',
       ylim=[1.8,2.8],xlim=[-.01,1.01],
       xticks=np.arange(.1,.91,.2),
       title=f'Minimal Energy-Volume curve for shell and crescent\n$\sigma_{{1,2}}, \sigma_{{1,3}}, \sigma_{{2,3}} = {σs[2]:.3f}, {σs[1]:.3f}, {σs[0]:.3f}$')
for axi in (ax,shape_ax):
    for spine in axi.spines:
        if spine == 'top' or spine == 'right':
            axi.spines[spine].set_visible(False)
axes[1].annotate('$(b)$',xy=(-.1,1.1),xycoords='axes fraction',fontsize=12)
            
fig.savefig(f'{plot_dir}/ch-shape-energy-comparison.pdf',bbox_inches='tight')

# Experiment vs simulation comparison plus schematic

In [ ]:
img_names = sorted(glob.glob('shared-files/experiments/2022-11-11/*.png'))

imgs = [plt.imread(name) for name in img_names]

differences = [True] + [np.any(np.abs(imgs[i] - imgs[i-1]) > .5) for i in range(1,len(imgs))]

imgs = [img for img, diff in zip(imgs,differences) if diff]

sim_figs = {i: sorted(glob.glob(f'hpc/conservation/vtks/pngs/chs-3D-comparison/chs-3D-comparison-?-00{i}*png')) for i in range(3)}
sim_imgs = {i: [plt.imread(name) for name in sim_figs[i]] for i in range(3)}

h, w, _ = sim_imgs[0][0].shape
d = 300
sel = np.s_[h//2-d:h//2+d, w//2-d:w//2+d]

sim_times = np.array([0.01*i for i in range(51)]
                    + [.5 + 0.1*i for i in range(46)]
                    + [5 + 1.*i for i in range(4)])
# sim_frames = [np.argmin(np.abs(sim_times - 0.1*i)) for i in range(20)][:len(frames)] #np.arange(0,38,6)[:len(frames)]

In [ ]:
# frames = [15*i for i in range(4)] + [70,100]#list(np.arange(0,80,15))
frames = [0,10,20,40,60,80]
sim_frames = [0,5,10,20,66,100]#[np.argmin(np.abs(sim_times - 0.1*i)) for i in range(20)][:len(frames)] #np.arange(0,38,6)[:len(frames)]
nf = len(frames)
plot_sim = 0
fig, ax = plt.subplots(2,nf,gridspec_kw={'wspace':-0.01,'hspace':-0.05},figsize=(2*nf,4))
for i, fr in enumerate(frames):
    img = imgs[fr][40:,40:]
    ax[0,i].imshow(img,extent=[0,1,0,1])
    ax[0,i].set(xticks=[],yticks=[],title=f'$T = {fr/2:.0f}$ min')
for i, fr in enumerate(sim_frames):
    ax[1,i].imshow(sim_imgs[plot_sim][fr][sel])
    ax[1,i].set(xticks=[],yticks=[])
    ax[1,i].set_xlabel(f'$t={sim_times[fr]:.2f}$',fontsize=12)
ax[0,0].set_ylabel(f'Experiment',fontsize=14)
ax[1,0].set_ylabel(f'Simulation',fontsize=14)
fig.set_facecolor('w')
fig.suptitle('Experimental vs simulated liquid-liquid phase separation',y=1,fontsize=16)
# fig.text(0.1,.96,'$(a)$',fontsize=16)
plt.savefig('hpc/conservation/figures/final/experiment-vs-sim-big.png',dpi=400,bbox_inches='tight')

# Combining experiment and schematic

In [ ]:
im1name = 'hpc/conservation/figures/final/experiment-vs-sim-big.png'
im2name = 'figures/ternary-plots/crescent-schematic.png'
im1 = plt.imread(im1name)
im2 = plt.imread(im2name)

fig, ax = plt.subplots(1,2,figsize=(9.5,3),gridspec_kw={'wspace':0,'width_ratios':[3,1]})
for axi in ax:
    axi.set(xticks=[],yticks=[])
    for spine in axi.spines:
        axi.spines[spine].set_visible(False)
ax[0].imshow(im1)
ax[1].imshow(im2)
ax[0].text(0,.93,'$(a)$',transform=ax[0].transAxes)
ax[1].text(0,.93,'$(b)$',transform=ax[1].transAxes)
fig.set_facecolor('w')
plt.savefig('hpc/conservation/figures/final/exp-sim-diagram.png',dpi=400,bbox_inches='tight')

# 3D simulation comparison

In [ ]:
sim_figs = {i: sorted(glob.glob(f'hpc/conservation/vtks/pngs/chs-3D-comparison/chs-3D-comparison-?-00{2-i}*png')) for i in range(3)}
sim_figs[-1] = sorted(glob.glob(f'hpc/conservation/vtks/pngs/chs-3D-comparison/ch-3D-comparison-?-000*png'))
sim_imgs = {i: [plt.imread(name) for name in sim_figs[i]] for i in range(-1,3)}

In [ ]:
h, w, _ = sim_imgs[0][0].shape
d = 300
sel = np.s_[h//2-d:h//2+d, w//2-d:w//2+d]

sim_times = np.array([0.01*i for i in range(51)]
                    + [.5 + 0.1*i for i in range(46)]
                    + [5 + 1.*i for i in range(4)])
sim_times_ch = np.array([0.01*i for i in range(51)]
                    + [.5 + 0.1*i for i in range(30)]
                    + [3.4 + 1.*i for i in range(4)])
# sim_frames = [np.argmin(np.abs(sim_times - 0.1*i)) for i in range(20)][:len(frames)] #np.arange(0,38,6)[:len(frames)]

In [ ]:
frames = [15*i for i in range(4)] + [70,100]#list(np.arange(0,80,15))
times = [0,.02,.05,.1,.2,.5,.7,1,1.5,2,4,6]
plot_sims = [-1,0,1,2]
sim_frames = {i: [np.argmin(np.abs((sim_times if i > -1 else sim_times_ch) - t)) for t in times] for i in sim_figs}
nf = len(sim_frames[0])
ns = len(plot_sims)
fig, ax = plt.subplots(ns,nf,gridspec_kw={'wspace':-0.01,'hspace':-0.05},figsize=(nf,ns))
for j, sim in enumerate(plot_sims):
    for i, fr in enumerate(sim_frames[sim]):
        if fr < len(sim_imgs[sim]):
            ax[j,i].imshow(sim_imgs[sim][fr][sel])
        ax[j,i].set(xticks=[],yticks=[])
        ax[j,i].set_xlabel(f'$t={sim_times[fr]:.2f}$',fontsize=12)
        for spine in ax[j,i].spines:
            ax[j,i].spines[spine].set_visible(False)

ax[0,0].set_ylabel(f'CH',fontsize=12)
ax[1,0].set_ylabel(f'CHS',fontsize=12)
ax[2,0].set_ylabel(f'CHSB $z$',fontsize=12)
ax[3,0].set_ylabel(f'CHSB $x$',fontsize=12)
fig.set_facecolor('w')
fig.suptitle('3D comparison of CH, CHS, CHSB $z$, CHSB $x$ models',y=.95,fontsize=12)
plt.savefig('hpc/conservation/figures/final/3D-sims.png',dpi=400,bbox_inches='tight')

# 3D simulation movie

In [ ]:
sim_figs = {i: sorted(glob.glob(f'hpc/conservation/vtks/pngs/chs-3D-comparison/chs-3D-comparison-?-00{2-i}*png')) for i in range(3)}
sim_imgs = {i: [plt.imread(name) for name in sim_figs[i]] for i in range(3)}

h, w, _ = sim_imgs[0][0].shape
d = 300
sel = np.s_[h//2-d:h//2+d, w//2-d:w//2+d]

sim_times = np.array([0.01*i for i in range(51)]
                    + [.5 + 0.1*i for i in range(46)]
                    + [5 + 1.*i for i in range(4)])

In [ ]:
nf = max(len(figs) for sim, figs in sim_figs.items())
ns = len(plot_sims)
for it in range(nf):
    fig, ax = plt.subplots(1,ns,gridspec_kw={'wspace':-0.01,'hspace':-0.05},figsize=(2*ns,2))
    for sim in plot_sims:
        if it < len(sim_imgs[sim]):
            ax[sim].imshow(sim_imgs[sim][it][sel])
        ax[sim].set(xticks=[],yticks=[])
        for spine in ax[sim].spines:
            ax[sim].spines[spine].set_visible(False)
    ax[0].set_xlabel(f'CHS',fontsize=12)
    ax[1].set_xlabel(f'CHSB $z$',fontsize=12)
    ax[2].set_xlabel(f'CHSB $x$',fontsize=12)
    fig.set_facecolor('w')
    fig.suptitle(f'3D comparison of CHS, CHSB $z$, CHSB $x$ models\n$t = {sim_times[it]:.2f}$',y=1,fontsize=12)
    plt.savefig(f'hpc/conservation/figures/final/3D-movie/3D-movie-{it:0>3d}.png',dpi=300,bbox_inches='tight')
    print(it)
    plt.close()
    

# Velocity plots

# Very small $\sigma_{1,2}$

In [ ]:
# figure measurements
θ2 = 20*np.pi/180
θ1 = np.pi-θ2 + 10**np.linspace(-5,-1,9)
θ3 = 2*np.pi - θ1 - θ2
θs = np.array([θ1,θ2+0*θ1,θ3]).T
σss = np.array([angle_to_surf([θ1[i],θ2,θ3[i]]) for i in range(len(θ1))])


In [ ]:
σss[:,::-1]

In [ ]:
pd.DataFrame(σss[:,::-1],columns=['σ12','σ13','σ23']).to_csv('small-surface-tensions.csv',index=False)

In [ ]:
σs = pd.read_csv('hpc/conservation/small-surface-tensions.csv')

In [ ]:
list(σs['σ12'])